In [1]:
import pandas as pd 

from experiment import Experiment
from api_tools import (FixedGPTPrompt, 
                       FixedPassiveGPTPrompt, 
                       FixedT5Prompt, 
                       FixedPassiveT5Prompt, 
                       run_ai21_prompt, 
                       run_gpt_prompt, 
                       run_t5_prompt)

from metrics import accuracy_report


# Object Control
Starting with the object control experiments, we will look at how 4 LMs do:
1. GPT3
2. T5 for QA 
3. Jurassic large 
4. Jurassic Jumbo

## Experimental settings
We're looking at different name pairs to control for gender. We have 2 male-female name pairs, 1 male-male pair, 1 female-female pair, and 1 neutral-neutral. 
We're also looking at 10 object control verbs and 5 different actions. 

In [2]:
# names = [("Tom", "Mary"), ("Tom", "Bill"), ("Mary", "Bill"), ("Ellen", "Mary"), ("Morgan", "Jaime"), ("Iago", "Hamlet"), ("Jules", "Yves"), ("Kurt", "Lena")]
names = [("Tom", "Mary"), ("Tom", "Bill"), ("Mary", "Bill"), ("Ellen", "Mary"), ("Morgan", "Jaime")]
verbs = ["told", "ordered", "called upon", "reminded", "urged", "asked", "persuaded", "convinced", "forced", "pushed"]
actions = [("to leave", "left"), ("to call home", "called home"), ("to reply", "replied"), ("to wipe the counter", "wiped the counter"), ("to dance", "danced")]
correct_index = 1


## GPT 3
For GPT3, inference is not deterministic, so we're running 5 replicants per prompt 

In [ ]:

# gpt_kwargs = {"max_tokens": 2, "temperature": 0.0}
# gpt_object_control_experiment  = Experiment("gpt3", "object-control", FixedGPTPrompt, run_gpt_prompt, 5, gpt_kwargs)

# gpt_object_control_experiment.run(names, correct_index, verbs, actions)

In [3]:
# gpt_df = gpt_object_control_experiment.format_results()

# gpt_df.to_csv("/Users/Elias/child-lm/results/gpt_object_control_swap_names.csv")

gpt_df = pd.read_csv("/Users/Elias/child-lm/results/gpt_object_control_swap_names.csv")

In [4]:
accuracy_report(gpt_df)

((0.562, 500),
 {'Mary,Mary': -1,
  'Mary,Bill': (0.42, 100),
  'Mary,Jaime': -1,
  'Morgan,Mary': -1,
  'Morgan,Bill': -1,
  'Morgan,Jaime': (0.0, 100),
  'Ellen,Mary': (0.93, 100),
  'Ellen,Bill': -1,
  'Ellen,Jaime': -1,
  'Tom,Mary': (0.81, 100),
  'Tom,Bill': (0.65, 100),
  'Tom,Jaime': -1},
 {'to call home': (0.56, 100),
  'to wipe the counter': (0.55, 100),
  'to dance': (0.59, 100),
  'to leave': (0.68, 100),
  'to reply': (0.43, 100)},
 {'forced': (0.6, 50),
  'reminded': (0.62, 50),
  'told': (0.64, 50),
  'urged': (0.66, 50),
  'ordered': (0.48, 50),
  'persuaded': (0.54, 50),
  'pushed': (0.56, 50),
  'convinced': (0.58, 50),
  'asked': (0.52, 50),
  'called upon': (0.42, 50)},
 {'to call home,forced': (0.6, 10),
  'to call home,reminded': (0.6, 10),
  'to call home,told': (0.6, 10),
  'to call home,urged': (0.7, 10),
  'to call home,ordered': (0.5, 10),
  'to call home,persuaded': (0.5, 10),
  'to call home,pushed': (0.6, 10),
  'to call home,convinced': (0.5, 10),
  'to c

## T5 for QA

In [3]:

# t5_object_control_experiment  = Experiment("t5", "object-control", FixedT5Prompt, run_t5_prompt, 1, None)

# t5_object_control_experiment.run(names, correct_index, verbs, actions, do_swap = False)

100%|██████████| 5/5 [04:47<00:00, 57.53s/it]


In [4]:
# t5_df = t5_object_control_experiment.format_results()

# t5_df.to_csv("/Users/Elias/child-lm/results/t5_object_control.csv")

t5_df = pd.read_csv("/Users/Elias/child-lm/results/t5_object_control.csv")

In [5]:
correct = t5_df[t5_df['true'] == t5_df['pred']]
incorrect = t5_df[t5_df['true'] != t5_df['pred']]

print(f"accuraccy: {(len(correct)/ len(t5_df)) * 100:.2f}")

accuraccy: 98.40


## Jurassic Large

In [ ]:

# jurassic_kwargs = {"maxTokens": 2, "temperature": 0.0}
# jurassic_object_control_experiment  = Experiment("jurassic-large", "object-control", FixedGPTPrompt, run_ai21_prompt, 1, jurassic_kwargs)

# jurassic_object_control_experiment.run(names, correct_index, verbs, actions)

In [6]:
# jurassic_df = jurassic_object_control_experiment.format_results()

# jurassic_df.to_csv("/Users/Elias/child-lm/results/jurassic_object_control_swap_names.csv")

jurassic_df = pd.read_csv("/Users/Elias/child-lm/results/jurassic_object_control_swap_names.csv")

In [7]:
accuracy_report(jurassic_df)




((0.26, 500),
 {'Mary,Mary': -1,
  'Mary,Bill': (0.49, 100),
  'Mary,Jaime': -1,
  'Morgan,Mary': -1,
  'Morgan,Bill': -1,
  'Morgan,Jaime': (0.0, 100),
  'Ellen,Mary': (0.26, 100),
  'Ellen,Bill': -1,
  'Ellen,Jaime': -1,
  'Tom,Mary': (0.35, 100),
  'Tom,Bill': (0.2, 100),
  'Tom,Jaime': -1},
 {'to call home': (0.18, 100),
  'to wipe the counter': (0.34, 100),
  'to dance': (0.2, 100),
  'to leave': (0.31, 100),
  'to reply': (0.27, 100)},
 {'forced': (0.28, 50),
  'reminded': (0.18, 50),
  'told': (0.26, 50),
  'urged': (0.28, 50),
  'ordered': (0.24, 50),
  'persuaded': (0.18, 50),
  'pushed': (0.26, 50),
  'convinced': (0.28, 50),
  'asked': (0.36, 50),
  'called upon': (0.28, 50)},
 {'to call home,forced': (0.3, 10),
  'to call home,reminded': (0.1, 10),
  'to call home,told': (0.2, 10),
  'to call home,urged': (0.2, 10),
  'to call home,ordered': (0.2, 10),
  'to call home,persuaded': (0.1, 10),
  'to call home,pushed': (0.1, 10),
  'to call home,convinced': (0.2, 10),
  'to cal

# Passives 

The passive form reverses the order and also syntactic role of the agent/patient. E.g. 

- Mary told Tom to wipe the counter 
- Tom was told by Mary to wipe the counter 

Now Tom is linearly further away from "wipe" and also the grammatical subject of the matrix clause (though still the patient) 

In [3]:
correct_index = 0 

## GPT3 

In [4]:

# gpt_kwargs = {"max_tokens": 2, "temperature": 0.0}
# passive_gpt_object_control_experiment  = Experiment("gpt3", "object-control-passive", FixedPassiveGPTPrompt, run_gpt_prompt, 5, gpt_kwargs) 

# passive_gpt_object_control_experiment.run(names, correct_index, verbs, actions)

# passive_gpt_df = passive_gpt_object_control_experiment.format_results()

# passive_gpt_df.to_csv("/Users/Elias/child-lm/results/gpt_passive_object_control_swap_names.csv")
passive_gpt_df = pd.from_csv("/Users/Elias/child-lm/results/gpt_passive_object_control_swap_names.csv")



100%|██████████| 5/5 [09:42<00:00, 116.59s/it]


In [5]:
accuracy_report(passive_gpt_df)

((0.454, 500),
 {'Mary,Mary': -1,
  'Mary,Jaime': -1,
  'Mary,Bill': (0.81, 100),
  'Tom,Mary': (0.34, 100),
  'Tom,Jaime': -1,
  'Tom,Bill': (0.67, 100),
  'Morgan,Mary': -1,
  'Morgan,Jaime': (0.45, 100),
  'Morgan,Bill': -1,
  'Ellen,Mary': (0.0, 100),
  'Ellen,Jaime': -1,
  'Ellen,Bill': -1},
 {'to call home': (0.5, 100),
  'to reply': (0.34, 100),
  'to wipe the counter': (0.57, 100),
  'to leave': (0.4, 100),
  'to dance': (0.46, 100)},
 {'urged': (0.46, 50),
  'asked': (0.38, 50),
  'persuaded': (0.38, 50),
  'convinced': (0.48, 50),
  'forced': (0.48, 50),
  'called upon': (0.44, 50),
  'ordered': (0.44, 50),
  'reminded': (0.44, 50),
  'pushed': (0.52, 50),
  'told': (0.52, 50)},
 {'to call home,urged': (0.5, 10),
  'to call home,asked': (0.4, 10),
  'to call home,persuaded': (0.4, 10),
  'to call home,convinced': (0.5, 10),
  'to call home,forced': (0.6, 10),
  'to call home,called upon': (0.6, 10),
  'to call home,ordered': (0.5, 10),
  'to call home,reminded': (0.4, 10),
  


## T5 for QA

In [4]:

passive_t5_object_control_experiment  = Experiment("t5", "object-control-passive", FixedPassiveT5Prompt, run_t5_prompt, 1, None)

passive_t5_object_control_experiment.run(names, correct_index, verbs, actions, do_swap = False)

passive_t5_df = passive_t5_object_control_experiment.format_results()

passive_t5_df.to_csv("/Users/Elias/child-lm/results/t5_passive_object_control.csv")

 40%|████      | 2/5 [01:21<02:14, 44.90s/it]

In [ ]:
accuracy_report(passive_t5_df)

## Jurassic Large 

In [ ]:

jurassic_kwargs = {"maxTokens": 2, "temperature": 0.0}
passive_jurassic_object_control_experiment  = Experiment("jurassic-large", "object-control-passive", FixedPassiveGPTPrompt, run_ai21_prompt, 1, jurassic_kwargs)

passive_jurassic_object_control_experiment.run(names, correct_index, verbs, actions)
passive_jurassic_df = passive_jurassic_object_control_experiment.format_results()

passive_jurassic_df.to_csv("/Users/Elias/child-lm/results/jurassic_passive_object_control_swap_names.csv")

# jurassic_df = pd.read_csv("/Users/Elias/child-lm/results/jurassic_object_control_swap_names.csv")

In [ ]:
accuracy_report(passive_t5_df)